In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import datetime as dt
from tqdm import tqdm
import pymorphy3

### Создадим функцию для выгрузки 77 названий вузов Санкт-Петербурга

In [390]:
base_url = 'https://vuzopedia.ru/region/city/50?page='
def parse_page(page_number):
    url = base_url + str(page_number)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Парсинг карточек вузов
    cards = soup.find_all('div', class_='col-md-7 blockAfter')
    unis = []
    
    for card in cards:
        title = card.find('div', class_='itemVuzTitle').text
        unis.append(title)
    
    return unis

### Выгрузим названия вузов с каждой из 7 страниц Вузопедии

In [32]:
spb_unis = []
for page in range(1, 8):  # Парсим 7 страниц
    unis = parse_page(page)
    spb_unis.extend(unis)

### Очистим названия вузов от лишних символов

In [33]:
for i in range(len(spb_unis)):
    spb_unis[i] = spb_unis[i].replace('  ', '').replace('\n', '')

In [34]:
spb_unis

['Балтийский государственный технический университет «ВОЕНМЕХ» имени Д. Ф. Устинова',
 'Санкт-Петербургский государственный электротехнический университет «ЛЭТИ» им. В.И. Ульянов..',
 'Санкт-Петербургский государственный университет телекоммуникаций им. проф. М.А.Бонч-Бруевича',
 'Санкт-Петербургский Гуманитарный университет профсоюзов',
 'Санкт-Петербургский государственный экономический университет',
 'Национальный исследовательский университет «Высшая школа экономики» — Санкт-Петербург',
 'РАНХиГС Санкт-Петербург',
 'Санкт-Петербургский университет технологий управления и экономики',
 'Санкт-Петербургский Горный университет',
 'Государственный университет морского и речного флота имени адмирала С. О. Макарова',
 'Санкт-Петербургский государственный аграрный университет',
 'Санкт-Петербургский государственный университет ветеринарной медицины',
 'Балтийская академия туризма и предпринимательства',
 'Санкт-Петербургский университет Министерства внутренних дел РФ',
 'Санкт-Петербургски

### Найдем филиалы вне города Санкт-Петербург

In [35]:
filtered_list = [s for s in spb_unis if 'филиал' in s]

In [43]:
filtered_list[-3]

'Волховский филиал Российского государственного педагогического университета им. Герцена'

In [45]:
filtered_list[-2]

'Ивангородский гуманитарно-технический институт (филиал) ГУАП'

### Удалим 2 филиала в Ленобласти за чертой города

In [46]:
del spb_unis[spb_unis.index(filtered_list[-3])]

In [47]:
del spb_unis[spb_unis.index(filtered_list[-2])]

In [50]:
spb_unis

['Балтийский государственный технический университет «ВОЕНМЕХ» имени Д. Ф. Устинова',
 'Санкт-Петербургский государственный электротехнический университет «ЛЭТИ» им. В.И. Ульянов..',
 'Санкт-Петербургский государственный университет телекоммуникаций им. проф. М.А.Бонч-Бруевича',
 'Санкт-Петербургский Гуманитарный университет профсоюзов',
 'Санкт-Петербургский государственный экономический университет',
 'Национальный исследовательский университет «Высшая школа экономики» — Санкт-Петербург',
 'РАНХиГС Санкт-Петербург',
 'Санкт-Петербургский университет технологий управления и экономики',
 'Санкт-Петербургский Горный университет',
 'Государственный университет морского и речного флота имени адмирала С. О. Макарова',
 'Санкт-Петербургский государственный аграрный университет',
 'Санкт-Петербургский государственный университет ветеринарной медицины',
 'Балтийская академия туризма и предпринимательства',
 'Санкт-Петербургский университет Министерства внутренних дел РФ',
 'Санкт-Петербургски

## Задача
1. Для каждого из 75 вузов найти их группу в ВК с новостями и мероприятиями
2. Выгрузить новости за последний год
3. Найти среди новостей упоминания о мероприятиях с ключевыми словами "толерантность", "терпимость", "экстремизм", "национальность", "религия", "уважение" ...
4. Вывести список всех универов, количество целевых мероприятий за 1 год и спосок ссылок на посты про каждое мероприятие

### Найдем сайты вузов через **Google Custom Search API**

Чтобы не компрометировать чувствительные данные, запишем **ключ API** в отдельном файле и считаем его в память

In [42]:
with open("C:/Users/zinov/Проекты/Толерантность/google_api_key.txt") as f:
  API_KEY = f.read()

In [43]:
with open("C:/Users/zinov/Проекты/Толерантность/google_id.txt") as f:
  SEARCH_ENGINE_ID = f.read()

Напишем фунцию для работы с **Google API**

In [44]:
def search_api(query):
    url = 'https://www.googleapis.com/customsearch/v1'
    params = {
        'q': query,
        'key': API_KEY,
        'cx': SEARCH_ENGINE_ID
    }
    response = requests.get(url, params=params)
    result = response.json()
    result = result['items'][0]['link']
    return result

In [139]:
sites = []

Запишем сайты в список

In [140]:
for uni in spb_unis:
    sites.append(search_api(uni))

### Проверим на ошибки и исправим ссылку на 1 вуз

In [143]:
index = sites.index('https://ru.wikipedia.org/wiki/%D0%92%D0%BE%D0%B5%D0%BD%D0%BD%D0%BE-%D0%BA%D0%BE%D1%81%D0%BC%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D0%B0%D0%BA%D0%B0%D0%B4%D0%B5%D0%BC%D0%B8%D1%8F_%D0%B8%D0%BC%D0%B5%D0%BD%D0%B8_%D0%90._%D0%A4._%D0%9C%D0%BE%D0%B6%D0%B0%D0%B9%D1%81%D0%BA%D0%BE%D0%B3%D0%BE')
sites[index] = 'https://vka.mil.ru/'

### Выделим сайты вузов **Минобороны**

In [192]:
for site in sites:
    if '.mil' in site:
        sites.append(site)
        sites.remove(site)

In [201]:
sites.append(sites[52])
del sites[52]

### С сайтов возьмем официальные группы ВКонтакте

In [162]:
def get_vk(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    vk = ''
    # Парсинг ссылки
    for link in soup.find_all('a', href=True):
        if 'vk.com' in link['href']:
            vk = link['href']
    
    return vk

In [203]:
vks = []
for site in tqdm(sites):
    vks.append(get_vk(site))

 85%|█████████████████████████████████████████████████████████████████████▉            | 64/75 [01:04<00:11,  1.00s/it]


SSLError: HTTPSConnectionPool(host='viit.vamto.mil.ru', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

### Создадим таблицу соответствия сайтов группам ВКонтакте

In [213]:
df = pd.DataFrame({'Сайт': sites[:64], 'ВК': vks})

### Для вузов **Минобороны** нужен сертификат Минцифры
Также некоторые сайты не имеют ссылок на группы ВКонтакте

In [386]:
sites[64:]

['https://viit.vamto.mil.ru/',
 'https://jdv.vamto.mil.ru/',
 'https://igps.ru/',
 'https://vamto.mil.ru/O_VUZe',
 'https://vas.mil.ru/',
 'https://vma.mil.ru/',
 'https://mvaa.mil.ru/',
 'https://vma.mil.ru/',
 'https://vka.mil.ru/',
 'https://spb.rpa-mu.ru/',
 'https://spbreaviz.ru/']

In [221]:
df[df.ВК == '']

,Сайт,ВК
2,https://www.sut.ru/,
4,https://unecon.ru/,
11,https://spbguvm.ru/,
18,https://www.spbgik.ru/,
23,https://spbu.ru/,
34,https://spvi.rosguard.gov.ru/,
50,https://spbrta.customs.gov.ru/,
57,https://www.vmeda.org/,
61,https://www.procuror.spb.ru/,


### Отработаем вручную

In [223]:
df['ВК'][2] = 'https://vk.com/sutru'
df['ВК'][4] = 'https://vk.com/unecon'
df['ВК'][11] = 'https://vk.com/spbguvm'
df['ВК'][18] = 'https://vk.com/spbgik_ru'
df['ВК'][23] = 'https://vk.com/spb1724'
df['ВК'][50] = 'https://vk.com/spbrta'
df['ВК'][61] = 'https://vk.com/studsovet_uprf'

In [283]:
df['ВК'][25] = 'https://vk.com/vshni_spb'

C:\Users\zinov\AppData\Local\Temp\ipykernel_8796\77171471.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['ВК'][25] = 'https://vk.com/vshni_spb'


### Из ссылок выделим короткие имена групп

In [241]:
def link2name(s):
    # Находим индекс символа
    index = s.rfind('/')
    # Делаем срез строки до найденного индекса
    if index != -1:
        s = s[index + 1:]
    return s

In [248]:
df['name'] = ''

### Запишем короткие имена в таблицу

In [249]:
for i in range(len(df.ВК)):
    df['name'][i] = link2name(df.ВК[i])

C:\Users\zinov\AppData\Local\Temp\ipykernel_8796\3943229371.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['name'][i] = link2name(df.ВК[i])
C:\Users\zinov\AppData\Local\Temp\ipykernel_8796\3943229371.py:2: FutureWarning: ChainedAssig

In [326]:
pd.set_option('display.max_rows', None)
df

,Сайт,ВК,name
0,https://www.voenmeh.ru/,https://vk.com/bstu_voenmeh,bstu_voenmeh
1,https://etu.ru/,https://vk.com/spbsetu,spbsetu
2,https://www.sut.ru/,https://vk.com/sutru,sutru
3,https://www.gup.ru/,https://vk.com/spbgup_official,spbgup_official
4,https://unecon.ru/,https://vk.com/unecon,unecon
5,https://spb.hse.ru/,https://vk.com/hse_spb,hse_spb
6,https://spb.ranepa.ru/spb-ranepa/,https://vk.com/spb.ranepa,spb.ranepa
7,https://www.spbume.ru/,https://vk.com/spbume_university,spbume_university
8,https://spmi.ru/,https://vk.com/spmi1773,spmi1773
9,http://gumrf.ru/,https://vk.com/gumrf,gumrf


### Напишем функцию для работы с **VK API**

In [348]:
def wallGet(owner_id, offset):
    version = '5.199'
    url = 'https://api.vk.com/method/wall.get?owner_id=' + str(owner_id) + '&count=100&offset=' + str(offset) + '&v=' + version + '&access_token=' +  token
    response = requests.get(url)
    return response.json().get('response')

Чтобы не компрометировать чувствительные данные, запишем **ключ API** в отдельном файле и считаем его в память

In [71]:
with open("C:/Users/zinov/Проекты/Толерантность/token.txt") as f:
  token = f.read()

### Напишем функцию для реструктуризации json-файла

In [61]:
def wall2dict(wall, dict):
    if wall != None:
        dict['post_id'].extend([post.get('id') for post in wall])      
        dict['text'].extend([post.get('text') for post in wall])         
        dict['likes'].extend([post.get('likes').get('count') if post.get('likes') != None else None for post in wall])         
        dict['comments'].extend([post.get('comments').get('count') if post.get('comments') != None else None for post in wall])       
        dict['views'].extend([post.get('views').get('count') if post.get('views') != None else None for post in wall])
        dict['date'].extend([post.get('date') for post in wall]) 

### Выгрузим по 100 постов из каждого сообщества
И запишем их в список walls

In [358]:
walls = []
for name in tqdm(df.name):
    if name == None:
        walls.append(pd.DataFrame())
        continue
    wall = wallGet(name, 0)
    if wall == None:
        walls.append(pd.DataFrame())
        continue
    wall = wall['items']
    posts = {
        'post_id': [],
        'text': [],
        'likes': [],
        'comments': [],
        'views': [],
        'date': []
    }
    wall2dict(wall, posts)
    wall_df = pd.DataFrame(posts)
    wall_df.date = pd.to_datetime(wall_df.date, unit = 's').dt.date
    walls.append(wall_df)

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [03:11<00:00,  2.99s/it]


### Объединим все посты в одну таблицу

In [391]:
df = pd.concat(walls, axis=0)

In [392]:
df = df.reset_index()

### Посчитаем количество целевых слов в постах

In [27]:
# Инициализация pymorphy3
morph = pymorphy3.MorphAnalyzer()

# Преобразование текста в нижний регистр для корректного подсчета
df['text'] = df['text'].str.lower().fillna('')

In [18]:
# Список слов для подсчета
words_to_count = ["толерантность", "терпимость", "экстремизм", "национальность", "религия", "уважение"]

# Приведение списка слов к нормальной форме
normalized_words_to_count = [morph.parse(word)[0].normal_form for word in words_to_count]

In [29]:
# Определение функции для приведения слова к нормальной форме
def normalize_word(word):
    return morph.parse(word)[0].normal_form

# Определение функции для подсчета упоминаний слов из списка с учетом словоформ
def count_words(text, words_list):
    words = text.split()
    normalized_words = [normalize_word(word) for word in words]
    return sum(normalized_words.count(word) for word in words_list)

In [30]:
# Применение функции к столбцу 'text' для подсчета всех слов из списка
df['total_count'] = df['text'].apply(lambda x: count_words(x, normalized_words_to_count))

In [403]:
df = df.drop(columns=['level_0', 'index', 'lower_text'])

### Сохраним промежуточный результат для дальнейшей работы

In [405]:
df.to_csv('spb_unis_walls.csv')

### Считаем полученный CSV-файл

In [21]:
df = pd.read_csv('spb_unis_walls.csv')

In [34]:
df = df.drop(columns=['tolerance_count'])

### Ознакомимся с полученными постами

In [41]:
for t in df[df.total_count > 0].text:
    print('-------------\n', t)

-------------
 свой день рождения сегодня отмечает петр михайлович винник – д.т.н., заведующий кафедрой о6 «высшая математика»! 

под руководством п.м. винника кафедра о6 активно развивается, внедряет в свою работу новейшие образовательные методики и достигает новых высот. петр михайлович – пример настоящего профессионала для своих многочисленных коллег и студентов. 

мы поздравляем петра михайловича с днем рождения и желаем ему уважения коллег и новых профессиональных успехов! крепкого здоровья, счастья и благополучия!
-------------
 📌как не ошибиться при выборе вуза? семь советов ректора спбгуп, академика а.с. запесоцкого
 
каждое лето сотни тысяч выпускников школ сдают единые государственные экзамены и поступают в вузы. как правило, большинство одиннадцатиклассников понимают, чем они хотят (или не хотят) заниматься в будущем, и уже составили для себя список вузов, где могли бы получить нужные знания и навыки. но самым сложным для абитуриента становится даже не сдача егэ, а финальный

# Выводы:
- Для дальнейшего анализа нужно изменить ключевые слова, так как многие посты, содержащие словоформы слова "уважение", оказались нерелевантными целям исследования